# Code of Conduct for Jupyter Notebook Template Usage

## Introduction

Welcome to our Jupyter Notebook environment. To ensure a productive and respectful environment, we have established a few ground rules. Please adhere to this Code of Conduct when using our Jupyter Notebook templates.

## Guidelines

### 1. **Notebook Structure**
   - This notebook has a specifc structure that should be respected and not to be tempered with.

### 2. **Integrity in Coding**
   - Write clear, commented, and understandable code.
   - Acknowledge and credit external sources or contributions.

### 3. **Data Privacy and Security**
   - Handle sensitive data responsibly.
   - Do not share confidential information without proper authorization.

### 4. **Collaboration and Sharing**
   - Collaborate effectively with others.
   - Share knowledge and help fellow users when possible.

### 5. **Responsible Resource Usage**
   - Use computational resources judiciously.
   - Avoid unnecessary computational tasks that can overload the system.

## Notebook Structure

This Notebook is split in three parts:
   - The First Part is a cell that gets and loads the data that was selected beforehand. (This cell needs only to be run, **not modified**.)
   - The Second Part is a cell in which the preparation for data is done. (This cell can be modified, but only to prepare the data in format for a specific ML model, not to make any other modifications on the data.)
   - The Third Part is compose of three cells. 
      - The first cell can be modified to prepare the model for training and train it.
      - The second cell can be modified to save the metrics in a json file, and the metrics shoud be saved as key value pair.
      - The third cell can't be modified, in this cell the saving of the model and metrics is realized.

   - Also at the begging of the notebook is a cell in which additional modules can be added, put only from standard library, **other modules can not be installed**.

## Reporting Issues

If you encounter any issues or observe violations of this Code of Conduct, please report them to [jarcau.stefan.cristian@gmail.com](jarcau.stefan.cristian@gmail.com).

## Conclusion

By adhering to these guidelines, we can maintain a healthy, productive, and welcoming environment for all users. Thank you for your cooperation and happy coding!


# Importing Modules

In [ ]:
import numpy as np
from transformers import AutoModelForObjectDetection, TrainingArguments, Trainer, AutoImageProcessor

# Loading the Data (First Run This Cell To Bring the Selected Data)

In [ ]:
import io
import os
import requests
import pandas as pd

url = os.getenv("DATASET_URL")
api = os.getenv("API")

response = requests.get(api + f"?path={url}")

if response.status_code == 200:
    df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))

df.to_csv("./temp_file.csv", index=False)

df.head()

# Data Preparation

In [ ]:
# Make neccesary Modifications on the df dataframe defined above, but do not remove columns or rows, just reshape it.

# Model Training and Testing (At the end of the last block specify model name)

In [ ]:
# In this cell any model from sklearn can be built the name of the variable that holds the model should be model

tokenizer = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")

model = AutoModelForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    ignore_mismatched_sizes=True,
)


training_args = TrainingArguments(
    output_dir="finetuned_model",
    per_device_train_batch_size=8,
    num_train_epochs=10,
    fp16=True,
    save_steps=200,
    logging_steps=50,
    learning_rate=1e-5,
    weight_decay=1e-4,
    save_total_limit=1,
    remove_unused_columns=False,
    push_to_hub=True # Change to false if you want not to push it to hub, if left to true please login to huggingface
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
# Login to huggingface if you want to have the model pushed there as well
!huggingface-cli login

# Set Metrics and Model Parameters

In [ ]:
#In this cell please put the metrics that result from the training and testing of the model inside the metrics dict (the metrics should be the key string and the value float), then run the cell
import json

metrics = {
    "": 0.0
}

params = {
    "": 0.0
}

model_name = ""

with open("metrics.json", "w") as metrics_file:
    json.dump(metrics, metrics_file, indent=4)

with open("params.json", "w") as params_file:
    json.dump(params, params_file, indent=4)

# Model Upload (!!!! Only run this cell once we have completed all the work !!!!)

In [ ]:
#This cell will save the model and close the connection, please refresh the page.
import subprocess

def is_variable_defined(var_name):
    return var_name in locals() or var_name in globals()

try:
    if model_name is not None and is_variable_defined("trainer") and is_variable_defined("model") and is_variable_defined("metrics") and is_variable_defined("params"):

        result = subprocess.run(['python3', '/home/noprem/upload_transformers_to_mlflow.py', f'-n {model_name}', f'-a {model.config.architectures[0]}'], capture_output=True, text=True)

        if result.stderr == "":
            if model_name:
                result_database = subprocess.run(['python3', '/home/noprem/save_to_postgres.py'], capture_output=True, text=True)
            if result_database.stderr == "":
                print("Success, Now you can close the page!")
            else:
                print("Please check your code and try again!")
        else:
            print("Please check your code and try again!")
except NameError as e:
    print("Model is not defined! Please check your code and try again!")
